# DNS ServerLevelPluginDll Install
Detects the installation of a plugin DLL via ServerLevelPluginDll parameter in Registry, which can be used to execute code in context of the DNS server (restart required)

## Rule Content
```
- action: global
  title: DNS ServerLevelPluginDll Install
  id: e61e8a88-59a9-451c-874e-70fcc9740d67
  status: experimental
  description: Detects the installation of a plugin DLL via ServerLevelPluginDll parameter
    in Registry, which can be used to execute code in context of the DNS server (restart
    required)
  references:
  - https://medium.com/@esnesenon/feature-not-bug-dnsadmin-to-dc-compromise-in-one-line-a0f779b8dc83
  date: 2017/05/08
  author: Florian Roth
  tags:
  - attack.defense_evasion
  - attack.t1073
  detection:
    condition: 1 of them
  fields:
  - EventID
  - CommandLine
  - ParentCommandLine
  - Image
  - User
  - TargetObject
  falsepositives:
  - unknown
  level: high
- logsource:
    product: windows
    service: sysmon
  detection:
    dnsregmod:
      EventID: 13
      TargetObject: '*\services\DNS\Parameters\ServerLevelPluginDll'
- logsource:
    category: process_creation
    product: windows
  detection:
    dnsadmin:
      CommandLine: dnscmd.exe /config /serverlevelplugindll *

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:*\\services\\DNS\\Parameters\\ServerLevelPluginDll)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:dnscmd.exe\ \/config\ \/serverlevelplugindll\ *')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()